In [ ]:
%run -i Scraping.py

In [2]:
df_complete = csv_2000_to_2019

In [4]:
import pandas as pd
import numpy as np

Se realizan distintas operaciones para ver la composición del dataframe

In [5]:
df_complete.shape

(1168, 7)